In [1]:
import scipy
from numpy import fft
import numpy as np
from numpy import linspace
import math
from scipy import integrate

In [2]:
eps=np.finfo(float).eps

In [3]:
eps

2.220446049250313e-16

In [4]:
# numerical grid
n = 2**13                   # number of grid points
twidth = 12.5e-12          # width of time window [s]
c = 299792458              # speed of light [m/s]
wavelength = 835e-9        # reference wavelength [m]
w0 = (2*np.pi*c)/wavelength   # reference frequency [Hz]
dt = twidth/n
T = np.arange(-n/2, n/2-1)*dt # time grid

# === input pulse
power = 10000                  # peak power of input [W]
t0 = 28.4e-15              # duration of input [s]
A = np.sqrt(power)*1/(np.cosh(T/t0)) # input field [W^(1/2)]

# === fibre parameters
flength = 0.15             # fibre length [m]
# betas = [beta2, beta3, ...] in units [s^2/m, s^3/m ...]
betas = [-1.1830e-026, 8.1038e-041, -9.5205e-056,  2.0737e-070,
         -5.3943e-085,  1.3486e-099, -2.5495e-114,  3.0524e-129, 
         -1.7140e-144]
gamma = 0.11               # nonlinear coefficient [1/W/m]
loss = 0                   # loss [dB/m]

# === Raman response
fr = 0.18                  # fractional Raman contribution
tau1 = 0.0122e-12; tau2 = 0.032e-12
RT = (tau1**2+tau2**2)/tau1/tau2**2*np.exp(-T/tau2)*np.sin(T/tau1)
RT = np.where(T < 0, RT, 0) # heaviside step function


# === simulation parameters
nsaves = 50     # number of length steps to save field at


In [26]:
# GNLSE Function
def gnlse(T, A, w0, gamma, betas, loss, fr, RT, flength, nsaves, printProgress = True):

    n = len(T)
    dT = T[1]-T[0]  # grid parameters
    V = (np.arange(-n/2,n/2)*2*np.pi)/(n*dT)  # frequency grid
    alpha = np.log(10**(loss/10))  # attenuation coefficient
    
    B = 0
    for i in range(len(betas)): # Taylor expansion of betas
        B += betas[i]/(math.factorial(i + 1)*V**(i + 1))
        
    L = 1j*B - alpha /2  # linear operator

    if abs(w0) > eps: # if w0>0 then include shock
        gamma = gamma/w0
        W=V+w0
    else: #set @ to 1 for no shock
        W = 1

    RW = n*fft.ifft(fft.fftshift(RT))  # frequency domain Raman
    L = fft.fftshift(L)
    W = fft.fftshift(W)  # shift to fft space

    def gnlse_rhs(z, AW):
        AT = fft.fft(AW*np.exp(L*z))  # time domain field
        AT = np.where(np.abs(AW) <= eps, AW, 0)
        IT = np.abs(AT)**2  # time domain intensity
        if (len(RT) == 1) or fr < eps:  # no Raman case
            M = fft.ifft(AT*IT)  # response function
        else:
            RS = dT*fr*fft.fft(fft.ifft(IT)*RW)  # Raman convolution
            M = fft.ifft(AT*((1-fr)*IT+RS))  # response function

        R = 1j*gamma*W*M*np.exp(-L*z)  # full RHS of Eq. (3.13)

        return R

    # === setup and run the ODE integrator
    Z = np.linspace(0, flength, nsaves)  # select output z points
    AW = []
    r = scipy.integrate.ode(gnlse_rhs).set_integrator("dopri5")  # choice of method
    r.set_initial_value(fft.ifft(A), 0) # initial values
    #help(r.integrate)
    for i in range(len(Z)):
        r.integrate(Z[i])#, AW) # get one more value, add it to the array
        if printProgress == True: 
            print(i + 1, "/", len(Z))
        AW.append(r.y)
        #if not r.successful():
        #    raise RuntimeError("Could not integrate")

    # === process output of integrator
    AT = []
    for i in range(len(AW)):
        AW[i] = AW[i] * np.exp(L*Z[i]) # change variables REMEMBER took ' out before Z
        AT.append(fft.ifft(AW[i]))           # time domain output
        AW[i] = fft.fftshift(AW[i])*dT*n  # scale

    W = V + w0  # the absolute frequency grid
    return {
            "AW" : AW, 
            "W" : W, 
            "AT": AT, 
            "Z" : Z
        }

In [27]:
# Multiple return values (tuple), explicitly unpack
#AW, W, AT = gnlse
# Multiple return values (tuple), keep tuple object, and index
#results = gnlse
#results[0] # AW
#results[1] # W
#results[2] # AT

In [28]:
# With dictonary
#results = gnlse
#results["AW"]

In [29]:
# propagate field

results = gnlse(T, A, w0, gamma, betas, loss, fr, RT, flength, nsaves)


1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50
50 / 50


In [30]:
results

{'AW': [array([ 4.06218028e-28+0.j,  4.97935204e-28+0.j, -1.99229738e-28+0.j, ...,
          1.09376065e-27+0.j, -3.52177708e-28+0.j,  2.99013783e-28+0.j]),
  array([-1.81802824e-22-3.19857692e-66j, -5.65403877e-23-9.94995320e-67j,
          2.19419996e-22+3.86228720e-66j, ...,
         -4.76826964e-21+8.39118049e-65j,  1.23196582e-21-2.16747867e-65j,
          1.81340764e-22+3.18966859e-66j]),
  array([-1.81802824e-22-6.39715384e-66j, -5.65403877e-23-1.98999064e-66j,
          2.19419996e-22+7.72457439e-66j, ...,
         -4.76826964e-21+1.67823610e-64j,  1.23196582e-21-4.33495734e-65j,
          1.81340764e-22+6.37933718e-66j]),
  array([-1.81802824e-22-9.59573076e-66j, -5.65403877e-23-2.98498596e-66j,
          2.19419996e-22+1.15868616e-65j, ...,
         -4.76826964e-21+2.51735415e-64j,  1.23196582e-21-6.50243602e-65j,
          1.81340764e-22+9.56900577e-66j]),
  array([-1.81802824e-22-1.27943077e-65j, -5.65403877e-23-3.97998128e-66j,
          2.19419996e-22+1.54491488e-65j, ...